In [33]:
import pandas as pd
def add_nda(utils_path,ndc_path):
    ndc = pd.read_csv(ndc_path)
    util = pd.read_csv(utils_path)
    print("Before")
    print("Number of rows in utlization is {0} with {1} unique drug names".format(util.shape[0],util.product_name.nunique()))
    ndc[['labeler_code','product_code']] = ndc['PRODUCTNDC'].str.split('-', 1, expand=True)
    ndc['labeler_code'] = [int(x) for x in ndc['labeler_code']]
    ndc['product_code'] = [int(x) for x in ndc['product_code']]
    ndc = ndc[ndc.MARKETINGCATEGORYNAME == "NDA"]
    results = util.merge(ndc[['labeler_code','product_code','APPLICATIONNUMBER']],on=['labeler_code','product_code'],how='left')
    results = results.dropna(subset={"APPLICATIONNUMBER"})
    results = results.rename(columns={"APPLICATIONNUMBER":"NDA"})
    print("After")
    print("Number of rows in utlization is {0} with {1} unique nda".format(results.shape[0],results.NDA.nunique()))
    return results

In [34]:
utlization_path = "../data/drug_utlization_data/full_du.csv"
ndc_path = "../data/ndcxls/product.csv"
utilization_nda = add_nda(utlization_path,ndc_path)
utilization_nda.head(2)

/Users/karunakargadireddy/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (3,8,18,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Before
Number of rows in utlization is 9141916 with 29638 unique drug names
After
Number of rows in utlization is 1139166 with 2117 unique nda


,utilization_type,state,labeler_code,product_code,package_size,year,quarter,product_name,suppression_used,units_reimbursed,...,medicaid_amount_reimbursed,non_medicaid_amount_reimbursed,quarter_begin,quarter_begin_date,latitude,longitude,location,ndc,supression_used,NDA
1,MCOU,NJ,75987.0,40,5.0,2017.0,4.0,Pennsaid,False,50.0,...,106421.25,0.0,10/1,10/01/2017 12:00:00 AM,41.1289,-98.2883,"(41.1289, -98.2883)",75987004005,NaN,NDA204623
6,FFSU,TN,338.0,89,4.0,2017.0,4.0,5% DEXTROS,True,NaN,...,NaN,NaN,10/1,10/01/2017 12:00:00 AM,41.6772,-71.5101,"(41.6772, -71.5101)",338008904,NaN,NDA016678


In [77]:
fares_dp = "../data/final_data/FAERS_Summary_NDA.csv"
fares_df = pd.read_csv(fares_dp)
print("Fares has {0} rows and {1} unique nda".format(fares_df.shape[0],fares_df.nda.nunique()))
fares_df = fares_df.rename(columns={"nda":"NDA"})
fares_df.head(2)

Fares has 164553 rows and 10362 unique nda


,NDA,year,quarter,adverse_affects
0,NDA-9223372036854775808,2015,1,1
1,NDA-9223372036854775808,2015,3,1


In [78]:
approval_dp = "../data/final_data/clean_approval_data.csv"
approval_df = pd.read_csv(approval_dp)
print("Approvals has {0} rows and {1} unique nda".format(approval_df.shape[0],approval_df.ID.nunique()))
approval_df = approval_df.rename(columns={"ID":"NDA"})
approval_df.head(2)

Approvals has 19256 rows and 18341 unique nda


,Approval Date,Drug Name,Submission,Active Ingredients,Company,Submission Classification *,Submission Status,year,month,NDA
0,1/1/2008,TIROSINTNDA #021924,SUPPL-1,LEVOTHYROXINE SODIUM,INSTITUT BIOCHIMIQUE,Labeling,Approval,2008,1,NDA021924
1,1/1/2020,PHOTOFRINNDA #020451,SUPPL-29,PORFIMER SODIUM,PINNACLE BIOLGS,Labeling,Approval,2020,1,NDA020451


In [79]:
#Merge Fares and utilization on nda,year,quarter
utilization_nda['year']=utilization_nda.year.astype('int64')
utilization_nda['quarter']=utilization_nda.quarter.astype('int64')
fares_df['year']=fares_df.year.astype('int64')
fares_df['quarter']=fares_df.quarter.astype('int64')
adverse_plus_utlization = utilization_nda.merge(fares_df,on= ["NDA","year","quarter"])
print("After merging fares and utlization")
print("Number of rows is {0} and number of unique nda is {1}".format(adverse_plus_utlization.shape[0],adverse_plus_utlization.NDA.nunique()))
adverse_plus_utlization.head(2)

After merging fares and utlization
Number of rows is 856866 and number of unique nda is 1802


,utilization_type,state,labeler_code,product_code,package_size,year,quarter,product_name,suppression_used,units_reimbursed,...,non_medicaid_amount_reimbursed,quarter_begin,quarter_begin_date,latitude,longitude,location,ndc,supression_used,NDA,adverse_affects
0,MCOU,NJ,75987.0,40,5.0,2017,4,Pennsaid,False,50.0,...,0.0,10/1,10/01/2017 12:00:00 AM,41.1289,-98.2883,"(41.1289, -98.2883)",75987004005,NaN,NDA204623,56
1,MCOU,VA,75987.0,40,5.0,2017,4,PENNSAID,False,36.0,...,0.0,10/1,10/01/2017 12:00:00 AM,35.7449,-86.7489,"(35.7449, -86.7489)",75987004005,NaN,NDA204623,56


In [80]:
#Merge Fares and approval dates
fares_df['year']=fares_df.year.astype('int64')
fares_df['quarter']=fares_df.quarter.astype('int64')
fares_plus_approval = fares_df.merge(approval_df,on= ["NDA"],how='inner')
print("After merging fares and approval")
print("Number of rows is {0} and number of unique nda is {1}".format(fares_plus_approval.shape[0],fares_plus_approval.NDA.nunique()))
fares_plus_approval.head(2)

After merging fares and approval
Number of rows is 160279 and number of unique nda is 8894


,NDA,year_x,quarter,adverse_affects,Approval Date,Drug Name,Submission,Active Ingredients,Company,Submission Classification *,Submission Status,year_y,month
0,NDA000793,2014,3,1,2/6/2019,BUTISOL SODIUMNDA #000793,SUPPL-26,BUTABARBITAL SODIUM,MYLAN SPECIALITY LP,Labeling,Approval,2019,2
1,NDA000793,2015,3,2,2/6/2019,BUTISOL SODIUMNDA #000793,SUPPL-26,BUTABARBITAL SODIUM,MYLAN SPECIALITY LP,Labeling,Approval,2019,2


In [81]:
#Merge utilization and approval dates
utilization_nda['year']=utilization_nda.year.astype('int64')
utilization_nda['quarter']=utilization_nda.quarter.astype('int64')
utilization_plus_approval = utilization_nda.merge(approval_df,on= ["NDA"],how='inner')
print("After merging approval and utlization")
print("Number of rows is {0} and number of unique nda is {1}".format(utilization_plus_approval.shape[0],utilization_plus_approval.NDA.nunique()))
utilization_plus_approval.head(2)

After merging approval and utlization
Number of rows is 1585342 and number of unique nda is 2099


,utilization_type,state,labeler_code,product_code,package_size,year_x,quarter,product_name,suppression_used,units_reimbursed,...,NDA,Approval Date,Drug Name,Submission,Active Ingredients,Company,Submission Classification *,Submission Status,year_y,month
0,MCOU,NJ,75987.0,40,5.0,2017,4,Pennsaid,False,50.0,...,NDA204623,1/16/2014,PENNSAIDNDA #204623,ORIG-1,DICLOFENAC SODIUM,HORIZON,Type 3 - New Dosage Form,Approval,2014,1
1,MCOU,TX,75987.0,40,5.0,2017,3,PENNSAID 2,False,264.0,...,NDA204623,1/16/2014,PENNSAIDNDA #204623,ORIG-1,DICLOFENAC SODIUM,HORIZON,Type 3 - New Dosage Form,Approval,2014,1


In [82]:
import regex as re
def clean_drug_name(df):
    try:
        drug_id = df['Drug Name'].split("#",1)[1]
        nda_split = df['Drug Name'].split("NDA",1)[0]
        if(' ' not in nda_split):
                drug_name = nda_split
                dosage = "None"
        elif('PLASTIC' in nda_split): 
                dosage = nda_split.split(' ',1)[1]
                drug_name = nda_split.split(' ',1)[0]
        else:
                drug_name = (re.split(r'(^[^\d]+)', nda_split)[1:])[0]
                dosage = (re.split(r'(^[^\d]+)', nda_split)[1:])[1]


        return(drug_name,dosage,drug_id)
    except:
        return

In [83]:
#Merge utilization and approval dates and fares
utilization_nda['year']=utilization_nda.year.astype('int64')
utilization_nda['quarter']=utilization_nda.quarter.astype('int64')
fares_df['year']=fares_df.year.astype('int64')
fares_df['quarter']=fares_df.quarter.astype('int64')
utilization_plus_approval = utilization_nda.merge(approval_df,on= ["NDA"],how='inner')
utilization_plus_approval = utilization_plus_approval.rename(columns={"year_x":"year"})
utilization_approval_fares = utilization_plus_approval.merge(fares_df,on=["NDA","year","quarter"])
utilization_approval_fares[['Drug Name', 'dosage', 'drug_id']] = utilization_approval_fares.apply(clean_drug_name, axis=1, result_type="expand")

print("After merging fares and utlization")
print("Number of rows is {0} and number of unique nda is {1}".format(utilization_approval_fares.shape[0],utilization_approval_fares.NDA.nunique()))
utilization_approval_fares.columns

After merging fares and utlization
Number of rows is 1162624 and number of unique nda is 1792


Index(['utilization_type', 'state', 'labeler_code', 'product_code',
       'package_size', 'year', 'quarter', 'product_name', 'suppression_used',
       'units_reimbursed', 'number_of_prescriptions',
       'total_amount_reimbursed', 'medicaid_amount_reimbursed',
       'non_medicaid_amount_reimbursed', 'quarter_begin', 'quarter_begin_date',
       'latitude', 'longitude', 'location', 'ndc', 'supression_used', 'NDA',
       'Approval Date', 'Drug Name', 'Submission', 'Active Ingredients',
       'Company', 'Submission Classification *', 'Submission Status', 'year_y',
       'month', 'adverse_affects', 'dosage', 'drug_id'],
      dtype='object')

In [84]:
#Read SIDER data
sider = pd.read_csv("../data/final_data/meddra_all_se.tsv",sep="\t",header=None)
drug_names = pd.read_csv("../data/final_data/drug_names.tsv",sep="\t",header=None)
sider.head(10)

,0,1,2,3,4,5
0,CID100000085,CID000010917,C0000729,LLT,C0000729,Abdominal cramps
1,CID100000085,CID000010917,C0000729,PT,C0000737,Abdominal pain
2,CID100000085,CID000010917,C0000737,LLT,C0000737,Abdominal pain
3,CID100000085,CID000010917,C0000737,PT,C0687713,Gastrointestinal pain
4,CID100000085,CID000010917,C0000737,PT,C0000737,Abdominal pain
5,CID100000085,CID000010917,C0002418,LLT,C0002418,Amblyopia
6,CID100000085,CID000010917,C0002418,PT,C0002418,Amblyopia
7,CID100000085,CID000010917,C0002871,LLT,C0002871,Anaemia
8,CID100000085,CID000010917,C0002871,PT,C0002871,Anaemia
9,CID100000085,CID000010917,C0003123,LLT,C0003123,Anorexia


,utilization_type,state,labeler_code,product_code,package_size,year,quarter,product_name,suppression_used,units_reimbursed,...,Submission,Active Ingredients,Company,Submission Classification *,Submission Status,year_y,month,adverse_affects,dosage,drug_id
